In [1]:
import random

import numpy as np

from operator import itemgetter

In [30]:
def point_mutation(bitstring, rate):
    '''
    Description:
        Randomly change a bit inside of a particular child

    Input:
        bitstring(list): list of bits
        rate(dbl): the rate at which the mutation happens

    Output:
        child(list): the bitstring with possible mutations from the original
    '''
    # Randomly change biots within the child 
    child = bitstring.copy()
    for idx, j in enumerate(bitstring):
        if random.uniform(0, 1) < rate:
            if child[idx] == 0:
                child[idx] = 1
            else:
                child[idx] = 0

    return child

def crossover(parent1, parent2, rate):
    '''
    Description:
        mutate a new child based off of the combination of two parents.

    Input:
        parent1(list): list of bitstrings
        parent2(list): list of bitstrings
        rate(dbl): the rate at which the muatation happens. If a random number
        is greater than the rate, no mutation will occur.

    Output:
        a possible combination of the two parents.
    '''
    # If the mutation rate is lower than the overall rate,
    # take part of parent 1 and part of parent two and combine them 
    # into a new child
    if random.uniform(0, 1) >= rate:
        return parent1
    else:
        point = 1+random.randint(0, (len(parent1)-2))
        return parent1[0:point]+parent2[point:len(parent2)]

def reproduce(selected, pop_size, p_cross, p_mutation):
    '''
    Description:
        Take parent strings a combine them combine/mutate them into children strings.

    Input:
        selected(list): list of dictionaries that are potential best strings
        pop_size(int): number of candiadtes to consider
        p_cross(dbl): rate at which the parent to parent crossover happens
        p_mutation(dbl): rate at which the single point mutation will happen

    Output:
        children(list): list of all the children created from the parents.
    '''
    children = []
    # Selects the items before (Odd indicies) and after (even indicies) 
    # for mutation later
    for i, p1 in enumerate(selected):
        if i % 2 == 0:
            p2 = selected[i+1]
        else:
            p2 = selected[i-1]

        if i == (len(selected)-1):
            p2 = selected[0]

        child = {}
        # Tries to combine the two parents into new children
        child['bitstring'] = crossover(p1['bitstring'], p2['bitstring'], p_cross)

        # Randomly mutates points within the child
        child['bitstring'] = point_mutation(child['bitstring'], p_mutation)
        children.append(child)
    
        if len(children) == pop_size:
            break

    return children

def binary_tournament(pop):
    '''
    Description:
        The function will randomly select two different lists from the population pool.
        It will then compared the cost of the two list against each other. Which ever
        list has a better cost, they get selected and move on in the selection process.

    Input:
        pop(list): List of dictionaries containing the cost and bits for each item under
        consideration

    Output:
        dictionary contating the list that won the "tournament"

        {bitstring: [],
         fitness: int}
    '''
    # Randomly select two bitstrings from the overall list
    i, j = random.randint(0, (len(pop)-1)), random.randint(0, (len(pop)-1))

    # Check to see if the indicies are the same
    while i == j:
        j =  random.randint(0, len(pop)-1)

    # Select the string with the highest cost to move forward.
    if pop[i]['fitness'] >= pop[j]['fitness']:
        return pop[i]
    else: 
        return pop[j]

def onemax(bitstring):
    '''
    The cost of the algorithm is the sum of the bits (Trying to obtain all 1's)
    '''
    return sum(bitstring)

def random_bitstring(num_bits):
    '''
    Description:
        Create a random list of bits (0 or 1)

    Input:
        num_bits(int): the number of bits contained in the list

    Output:
        A list of randomly generated bits.
    '''
    return [1 if random.uniform(0, 1) < .50 else 0 for i in range(num_bits)]

def search(max_gens, num_bits, pop_size, p_crossover, p_mutation):
'''
Description:
    This function contians the main logic of the program. It will search through
    the various combinations of children and parents to find a soultion that will
    optimize the given cost functions. In this case, the optimal solution is a 
    list of all ones.

Input:
    max_gens(int): The number of generations to loop through to find an optimal
    solution.
    num_bits(int): The number of bits contained in the list
    pop_size(int): The number of children to be children to be considered against the parent.
    p_crossover(dbl): THe probability of keeping the parent vs mixing aprent and child.
        ex) p_crossover = .99; probability of keeping parent to next generation = .01
    p_mutation(dbl): The probability of changing a single position in a list.

Output:
    best(list): A list of a possible optimal solution.  
'''
    # Create a random set of possible solutions
    population = [{'bitstring':random_bitstring(num_bits)} for i in range(pop_size)]
    
    # Find the "cost" of each colution
    for idx,_ in enumerate(population):
        population[idx]['fitness'] = onemax(population[idx]['bitstring'])

    # Sort the lists to find the solution with the highest cost, and keep the largest as the best solution
    best = sorted(population, key=itemgetter('fitness'), reverse=True)[0]

    # Loop through the number of generations
    for gen in range(max_gens):
        # Randomly select possible offspring
        selected = [binary_tournament(population) for _ in range(len(population))]
        # Reproduce children based off of the parent 
        children = reproduce(selected, pop_size, p_crossover, p_mutation)

        for idx,_ in enumerate(children):
            children[idx]['fitness'] = onemax(children[idx]['bitstring'])

        sort_children =  sorted(children, key=itemgetter('fitness'), reverse=True) 

        # If the best child has a higher cost than current, replace the current with child
        if sort_children[0]['fitness'] >= best['fitness']:
            best = sort_children[0].copy()

        population = sort_children.copy()
        print('Gen: {0}, Fitness: {1}, Best: {2} '.format(gen, best['fitness'], best['bitstring']))

        # Break out of the loop if the found the best solution before the end of the loop
        if sum(best['bitstring']) == num_bits:
            break

    return best

In [33]:
num_bits = 64
max_gens = 100
pop_size = 100
p_crossover = .99
p_mutation = 1.0/num_bits

best = search(max_gens, num_bits, pop_size, p_crossover, p_mutation)

Gen: 0, Fitness: 39, Best: [0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0] 
Gen: 1, Fitness: 44, Best: [1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1] 
Gen: 2, Fitness: 44, Best: [1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1] 
Gen: 3, Fitness: 44, Best: [0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0] 
Gen: 4, Fitness: 46, Best: [1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1